In [60]:
import psycopg2
import pandas as pd
from datetime import datetime

In [2]:
defects = [
    'pnd2_heat_deflection',
    'pnd3_pipe_damage'
    ]
sensor = "KKS_02SP_10_E901__XQ02"
tstart_pnd2 = 1594688912000
tend_pnd2 = 1594689920000

In [10]:
#Получение описания дефектов
conn_params = "dbname=block002catalog user=vti password=vti host=10.202.248."+host
sensor_result = []
sql = "SELECT def_id, def_name FROM defect;"
def_names = {}
with psycopg2.connect(conn_params) as conn:
        with conn.cursor() as cur:
            for defect in defects:
                cur.execute(sql)
                for i in cur.fetchall():
                    def_names[i[0]] = i[1]

In [77]:
#Получение статистики по продолжительности дефекта            
thres = [0, 60000, 180000, 600000, 1800000, 3600000, "FLOAT8 '+infinity'"]
levels = ['0 OR value = 1 OR value = 2 OR value = 3 OR value =4',0, 1, 2, 3, 4]
columns_names = ['<1_мин', '1-3_мин', '3-10_мин', '10-30_мин', '30-60_мин', '>60_мин']

sql =  'SELECT COUNT(difference) AS number\
        from\
        (SELECT  value, timestamp, -1*(timestamp-lead(timestamp)\
        OVER (ORDER BY timestamp)) AS difference\
        FROM def_{1}\
        WHERE value!= -1  AND timestamp >= {4} AND timestamp <= {5}) AS tab\
        WHERE difference > {2} AND difference < {0} AND (value = {3});'

conn_params = "dbname=state user=vti password=vti host=10.202.248."+host
with psycopg2.connect(conn_params) as conn:
        with conn.cursor() as cur:
            res = pd.DataFrame(columns = columns_names)
            df = pd.DataFrame(columns = columns_names )
            for level in levels:
                results = []
                for i in range(len(thres)-1):
#                     print(sql.format(thres[i+1],defect,thres[i]),'\n')
                    cur.execute(sql.format(thres[i+1],defects[0],thres[i], level, tstart_pnd2, tend_pnd2))
                    results.append(cur.fetchone()[0])
                df.loc[len(df)] = results
            df.index = [defect+': '+str(level) for level in levels]
#                 print(df)
            res = df if len(res)==0 else res.append(df)

In [73]:
def def_duration_analys(defect, tstart, tend, host = str(91)):

    conn_params = "dbname=asutp user=vti password=vti host=10.202.248."+host
    
    def get_date(time):
        return datetime.fromtimestamp(time/1000).strftime('%Y-%m-%d %H:%M:%S')

    sensor_result = []
    sql = "SELECT kks_timestamp, kks_value FROM {} WHERE kks_timestamp >= {}-5000 AND kks_timestamp <= {}+5000 ORDER BY kks_timestamp ;"
    
    #Получение значений датчика мощности
    with psycopg2.connect(conn_params) as conn:
            with conn.cursor() as cur:
                cur.execute(sql.format(sensor, tstart, tend))
                sensor_result = [i for i in cur.fetchall()]
    sensor_timestamps = [i[0] for i in sensor_result]
    sensor_values = [i[1] for i in sensor_result]

    #Получение данных о дефекте: степени развития, начале, продолжительности, скорости изменения мощности
    sql =  'SELECT value, timestamp, difference AS Продолжительность_дефекта\
            from\
            (SELECT  value, timestamp, -1*(timestamp-lead(timestamp)\
            OVER (ORDER BY timestamp)) AS difference\
            FROM def_{0}\
            WHERE value!= -1  AND timestamp >= {1} AND timestamp <= {2}) AS tab;'

    conn_params = "dbname=state user=vti password=vti host=10.202.248."+host
    with psycopg2.connect(conn_params) as conn:
            with conn.cursor() as cur:
                res = pd.DataFrame(columns = columns_names)
                df = pd.DataFrame(columns = columns_names )
                cur.execute(sql.format(defect, tstart, tend))
                results =cur.fetchall()

    for i in range(len(results)-1):
        start_idx = next(j-1 for j in range(len(sensor_timestamps)) if sensor_timestamps[j]>results[i][1])
        end_idx = next(j for j in range(len(sensor_timestamps)) if sensor_timestamps[j]>=results[i+1][1])
        vdN = (sensor_values[end_idx] - sensor_values[start_idx])/(sensor_timestamps[end_idx] - sensor_timestamps[start_idx]) * 1000 * 60
        vdN = round(vdN, 2)
        print('Дефект',results[i][0],'Начало',get_date(results[i][1]),'Конец',get_date(results[i+1][1]),'Скорость изменения', vdN)

In [76]:
def_data = [
    ['pnd2_heat_deflection', 1594688912000, 1594689920000],
    ['pnd3_condensate_undercooling', 1592205056000, 1592205087000]
]
for def_datum in def_data:
    print(def_datum)
    def_duration_analys(def_datum[0],def_datum[1],def_datum[2])

['pnd2_heat_deflection', 1594688912000, 1594689920000]
Дефект 2.0 Начало 2020-07-14 04:08:32 Конец 2020-07-14 04:09:37 Скорость изменения 5.61
Дефект 1.0 Начало 2020-07-14 04:09:37 Конец 2020-07-14 04:23:19 Скорость изменения 1.9
Дефект 3.0 Начало 2020-07-14 04:23:19 Конец 2020-07-14 04:23:45 Скорость изменения 4.33
Дефект 2.0 Начало 2020-07-14 04:23:45 Конец 2020-07-14 04:25:20 Скорость изменения 0.46
['pnd3_condensate_undercooling', 1592205056000, 1592205087000]
Дефект 3.0 Начало 2020-06-15 10:10:56 Конец 2020-06-15 10:11:22 Скорость изменения -4.72


In [79]:
def_data = [
    ['pnd3_pipe_damage', 1591658378000, 1591658414000],
    ['pnd3_pipe_damage', 1591739919000, 1591747326000],
    ['pnd3_pipe_damage', 1591758133000, 1591773943000],
    ['pnd3_pipe_damage', 1591934612000, 1591935083000]
]
for def_datum in def_data:
    print(def_datum)
    def_duration_analys(def_datum[0],def_datum[1],def_datum[2])

['pnd3_pipe_damage', 1591658378000, 1591658414000]
Дефект 3.0 Начало 2020-06-09 02:19:38 Конец 2020-06-09 02:19:40 Скорость изменения 2.06
Дефект 1.0 Начало 2020-06-09 02:19:40 Конец 2020-06-09 02:19:41 Скорость изменения -6.56
Дефект 0.0 Начало 2020-06-09 02:19:41 Конец 2020-06-09 02:20:11 Скорость изменения -8.38
Дефект 1.0 Начало 2020-06-09 02:20:11 Конец 2020-06-09 02:20:14 Скорость изменения -11.56
['pnd3_pipe_damage', 1591739919000, 1591747326000]
Дефект 2.0 Начало 2020-06-10 00:58:39 Конец 2020-06-10 00:58:42 Скорость изменения -7.19
Дефект 0.0 Начало 2020-06-10 00:58:42 Конец 2020-06-10 00:59:10 Скорость изменения 4.19
Дефект 1.0 Начало 2020-06-10 00:59:10 Конец 2020-06-10 00:59:12 Скорость изменения -1.41
Дефект 3.0 Начало 2020-06-10 00:59:12 Конец 2020-06-10 00:59:14 Скорость изменения 10.31
Дефект 4.0 Начало 2020-06-10 00:59:14 Конец 2020-06-10 01:56:17 Скорость изменения -0.17
Дефект 3.0 Начало 2020-06-10 01:56:17 Конец 2020-06-10 01:56:27 Скорость изменения 0.56
Дефект 1.0